In [23]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import os



In [24]:
def setup_driver():
    chrome_options =  webdriver.ChromeOptions()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')


    
    service = Service("C:/Users/mehdi/Documents/webdrivers/chromedriver.exe")
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

In [ ]:

rows_data = []
url = "https://sjg.eadl.ir/dashboard/manager/logs"
output_file = "activity_logs.xlsx"
# Load the initial page
driver.get(url)

In [25]:

# Get all rows except the header row (index 0)
rows = driver.find_elements(By.CSS_SELECTOR, "tr.ant-descriptions-row")[1:]

for row in rows:
    cells = row.find_elements(By.CSS_SELECTOR, "th.ant-descriptions-item-content span")
    row_data = [cell.text.strip() for cell in cells]
    if row_data:  # Only add non-empty rows
        rows_data.append(row_data)


In [26]:
def get_last_page_number(driver):
    try:
    last_page_link = driver.find_element(By.CSS_SELECTOR, "a[data-page]:last-of-type")
    return int(last_page_link.get_attribute('data-page'))
    except Exception as e:
    print(f"Error getting last page number: {e}")
    return None

In [ ]:
# def scrape_activity_logs():


try:

    
    # Get the last page number
    last_page = get_last_page_number(driver)
    if not last_page:
        print("Could not determine last page number")
        return
    
    print(f"Total pages to scrape: {last_page}")
    
    # Create DataFrame to store all data
    all_data = []
    
    # Loop through all pages
    current_page = 1
    while current_page <= last_page:
        print(f"Scraping page {current_page} of {last_page}")
        
        # Wait for table to be visible
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "tr.ant-descriptions-row"))
        )
        
        # Get data from current page
        page_data = get_table_data(driver)
        all_data.extend(page_data)
        
        # Save progress after each page (without headers)
        df = pd.DataFrame(all_data)
        df.to_excel(output_file, index=False, header=False)
        
        # Go to next page if not on last page
        if current_page < last_page:
            try:
                next_page = driver.find_element(By.CSS_SELECTOR, f"a[data-page='{current_page + 1}']")
                driver.execute_script("arguments[0].click();", next_page)
                time.sleep(2)  # Wait for page load
            except Exception as e:
                print(f"Error navigating to next page: {e}")
                break
        
        current_page += 1
    
    print(f"Scraping completed. Data saved to {output_file}")
    print(f"Total records scraped: {len(all_data)}")
    
except Exception as e:
    print(f"An error occurred: {e}")

finally:
    print('done')



Total pages to scrape: 2
Scraping page 1 of 2
Scraping page 2 of 2
Scraping completed. Data saved to activity_logs.xlsx
Total records scraped: 20
done
